# Yelp Dataset Recommendation

### Librerías

In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Lectura del dataset

In [67]:
data = []
with open("yelp_academic_dataset_business.json", encoding = "utf-8") as data_file:
    for line in data_file:
        data.append(json.loads(line))

yelp_full = pd.DataFrame(data)
print("Total businesses:", yelp_full.shape[0])
yelp_full.head()

Total businesses: 150346


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


### Selección de los restaurantes

In [68]:
Categories={}
yelp_full = yelp_full.dropna(subset=["categories"])
yelp_full["categories_splitted"] = yelp_full["categories"].apply(lambda x: [cat.strip() for cat in x.split(",")])
yelp_restaurants = yelp_full[yelp_full["categories_splitted"].apply(lambda cats: "Restaurants" in cats)]

In [69]:
print("Total Restaurants:", yelp_restaurants.shape[0])
yelp_restaurants.head()

Total Restaurants: 52268


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,categories_splitted
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","[Restaurants, Food, Bubble Tea, Coffee & Tea, ..."
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...","[Burgers, Fast Food, Sandwiches, Food, Ice Cre..."
8,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,"[Pubs, Restaurants, Italian, Bars, American (T..."
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers..."
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...","[Vietnamese, Food, Restaurants, Food Trucks]"


### Incorporación reviews

In [70]:
data = []
with open("yelp_academic_dataset_review.json", encoding = "utf-8") as data_file:
    for line in data_file:
        data.append(json.loads(line))
yelp_reviews = pd.DataFrame(data)

print("Total reviews:", yelp_reviews.shape[0])
yelp_reviews.head()

Total reviews: 6990280


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [71]:
yelp_reviews_grouped = yelp_reviews.groupby("business_id")["text"].apply(list).reset_index()
yelp = pd.merge(yelp_restaurants, yelp_reviews_grouped, on="business_id", how="left")
print("Total Restaurants:", yelp.shape[0])
yelp.head()

Total Restaurants: 52268


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,categories_splitted,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","[Restaurants, Food, Bubble Tea, Coffee & Tea, ...",[This is nice little Chinese bakery in the hea...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...","[Burgers, Fast Food, Sandwiches, Food, Ice Cre...",[I have always liked Sonic as it is good fresh...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",None,"[Pubs, Restaurants, Italian, Bars, American (T...","[I like Tsevis because they have good gyros, w..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...","[Ice Cream & Frozen Yogurt, Fast Food, Burgers...",[Waited several minutes waiting to order. I wa...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...","[Vietnamese, Food, Restaurants, Food Trucks]",[I eat pho about 4 times a week and from a spe...


### Preprocesamiento

In [72]:
del yelp_full
del yelp_restaurants
del yelp_reviews

#### Eliminar columnas innecesarias

In [ ]:
yelp = yelp.drop(columns=["is_open", "hours", "postal_code", "latitude", "longitude", "categories_splitted"])
print("Total Restaurants:", yelp.shape[0])
yelp.head()

Total Restaurants: 52268


,business_id,name,address,city,state,stars,review_count,attributes,categories,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",[This is nice little Chinese bakery in the hea...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...",[I have always liked Sonic as it is good fresh...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,3.0,19,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...","[I like Tsevis because they have good gyros, w..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,1.5,10,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",[Waited several minutes waiting to order. I wa...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks",[I eat pho about 4 times a week and from a spe...


#### Eliminar duplicados

In [74]:
print("IDs duplicados:", yelp.business_id.duplicated().sum())
print("Restaurantes duplicados (nombre, dirección, ciudad y estado)", yelp.loc[: , ["name", "address", "city", "state"]].duplicated().sum())

IDs duplicados: 0
Restaurantes duplicados (nombre, dirección, ciudad y estado) 43


In [75]:
yelp.drop_duplicates(subset = ["name", "address", "city", "state"], inplace=True)
print("Total Restaurants after deleting duplicates:", yelp.shape[0])
yelp.head()

Total Restaurants after deleting duplicates: 52225


,business_id,name,address,city,state,stars,review_count,attributes,categories,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",[This is nice little Chinese bakery in the hea...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...",[I have always liked Sonic as it is good fresh...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,3.0,19,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...","[I like Tsevis because they have good gyros, w..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,1.5,10,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",[Waited several minutes waiting to order. I wa...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks",[I eat pho about 4 times a week and from a spe...


### Preprocesado de texto

#### Conversión a minúsculas

In [76]:
yelp.text = yelp.text.apply(lambda x: " ".join(x) if isinstance(x, list) else x)
yelp.text = yelp.text.apply(lambda x: x.lower() if isinstance(x, str) else x)
yelp.head()

,business_id,name,address,city,state,stars,review_count,attributes,categories,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",this is nice little chinese bakery in the hear...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...",i have always liked sonic as it is good fresh ...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,3.0,19,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...","i like tsevis because they have good gyros, wh..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,1.5,10,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",waited several minutes waiting to order. i was...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks",i eat pho about 4 times a week and from a spec...


#### Eliminación de los signos de puntuación

In [77]:
import string
yelp.text = yelp.text.apply(lambda x: x.translate(str.maketrans("", "", string.punctuation)) if isinstance(x, str) else x)
yelp.head()

,business_id,name,address,city,state,stars,review_count,attributes,categories,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",this is nice little chinese bakery in the hear...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...",i have always liked sonic as it is good fresh ...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,3.0,19,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",i like tsevis because they have good gyros whi...
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,1.5,10,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",waited several minutes waiting to order i was ...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks",i eat pho about 4 times a week and from a spec...


#### Eliminación de las stopwords

In [78]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
yelp.text = yelp.text.apply(lambda x: " ".join(word for word in x.split() if word not in stop_words) if isinstance(x, str) else x)
yelp.head()

,business_id,name,address,city,state,stars,review_count,attributes,categories,text
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,4.0,80,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",nice little chinese bakery heart philadelphias...
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,2.0,6,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...",always liked sonic good fresh food far fast fo...
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,3.0,19,"{'Caters': 'True', 'Alcohol': 'u'full_bar'', '...","Pubs, Restaurants, Italian, Bars, American (Tr...",like tsevis good gyros really thing get ive br...
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,1.5,10,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...",waited several minutes waiting order car waiti...
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,4.0,10,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks",eat pho 4 times week specific restaurant tried...


### Obtención de las palabras más frecuentes


In [85]:
def get_top_words(column, top_n=10, ngram_range=(1, 1)):
    vectorizer = CountVectorizer(ngram_range=ngram_range, stop_words='english')
    bag_of_words = vectorizer.fit_transform(column)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
    words_freq = sorted(words_freq, key=lambda x: x[1], reverse=True)
    return words_freq[:top_n]


yelp = yelp.sample(frac = 0.3)

top_words = get_top_words(yelp.text, top_n=10, ngram_range=(1, 1))
top_words_df = pd.DataFrame(top_words, columns=['word', 'frequency'])
print(top_words_df)

      word  frequency
0     food    1020685
1     good     819588
2    place     713072
3    great     658698
4  service     495225
5     like     443319
6     time     408973
7   really     340407
8  ordered     324826
9  chicken     299519


### Vectorización TF-IDF

In [165]:
from sklearn.metrics.pairwise import cosine_similarity

yelp_tfidf = yelp.set_index("business_id", inplace = False)
indices = pd.Series(yelp_tfidf.index)

tfidf = TfidfVectorizer(analyzer = "word", ngram_range = (1, 1), min_df = 0.01, stop_words = "english")
tfidf_matrix = tfidf.fit_transform(yelp_tfidf.text)

cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

### Función de recomendación

In [172]:
def recommend(name, cosine_similarities = cosine_similarities):

    reccommendations = []

    id = yelp.loc[yelp.name == name, "business_id"].values[0]
    idx = indices[indices == id].index[0]

    score = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_indices = score.index[1:11]

    recs = yelp_tfidf.iloc[top_indices][["name", "categories", "stars", "review_count", "city", "state"]]
    recs = recs.sort_values(by="stars", ascending=False)

    print("Top %s de restaurantes similares a %s según las reseñas:" %(str(len(recs)), name))

    return recs

restaurant = yelp.name.sample(1).values[0]
recommend(restaurant)

Top 10 de restaurantes similares a Cedar Park Cafe según las reseñas:


,name,categories,stars,review_count,city,state
business_id,,,,,,
4caDWodT3H9v4ekwCsh6oQ,Sammy’s Bullfrog Cafe,"Breakfast & Brunch, Restaurants",5.0,87,Harleysville,PA
LdK6Tqr9-QK8eNezCUes9A,Mel's Kitchen,"Sandwiches, Soup, Cheesesteaks, Event Planning...",4.5,206,Philadelphia,PA
-qXHPxXkzTxYSelS4nLOhg,Bullard's American Cafe,"Breakfast & Brunch, American (New), Restaurant...",4.5,107,Warminster,PA
SA1kBystZZn8yYiQsjljYQ,Cafe Chorba,"Restaurants, American (New), Breakfast & Brunch",4.0,85,Morrisville,PA
dmx9rfhDOixCsCA_DcsEEA,Havertown Grille,"Breakfast & Brunch, Restaurants, Food, Coffee ...",4.0,115,Havertown,PA
ZwkdQGbg27T1jtFSP6-ggw,Gateway Cafe,"Breakfast & Brunch, Cafes, Restaurants",4.0,96,King of Prussia,PA
kpl3H_sgfTm-tH3Aa-ML7g,Benton Park Café & Coffee Bar,"Nightlife, Coffee & Tea, Beer, Wine & Spirits,...",4.0,533,Saint Louis,MO
TlVR0GNI6MsJyS0e74OhnA,Canopy Road Cafe - Tampa,"Restaurants, Breakfast & Brunch, Diners",4.0,252,Tampa,FL
C-_qY1Ptl7dHLJIKTwFY5w,Home Plate,"Restaurants, American (Traditional)",4.0,117,Dunedin,FL


In [ ]:
reccommendations = []

idx = indices[indices == name].index[0]

score = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

top_indices = score.index[0:11]

for i in top_indices:
    output_df = reccommendations.append(pd.DataFrame(yelp_tfidf[["categories", "stars", "review_count"]][yelp_tfidf.index == i]).sample())

    output_df = output_df.sort_values(by = "stars", ascending = False)

    print("Top %s de restaurantes similares a", name, "según las reseñas:" %(str(len(output_df)), name))

    return output_df

In [164]:
name = "Wawa"

yelp[yelp.name == name].head()

,business_id,name,address,city,state,stars,review_count,attributes,categories,text
15459,i9_2zZZAhQwzFe1Pk3QnXw,Wawa,1814 Salem Rd,Burlington,NJ,2.0,7,"{'RestaurantsPriceRange2': '1', 'GoodForKids':...","Delis, Restaurants, Food, Gas Stations, Conven...",food generally good well made deli staff rarel...
37559,khmaw-iyVP_y32ZWUwuwiA,Wawa,635 Chews Landing Rd,Lindenwold,NJ,4.0,9,"{'Caters': 'None', 'BikeParking': 'True', 'Alc...","Restaurants, Sandwiches, Coffee & Tea, Food, C...",ive many times service usually ok atms bad sin...
43184,kayVS-MPUVp_Vmo54Pjp9w,Wawa,193 Belmont Ave,Bala Cynwyd,PA,2.5,24,"{'RestaurantsReservations': 'False', 'BikePark...","Coffee & Tea, Food, Restaurants, Delis, Conven...",day hasnt wawa disappointed hope didnt jinx iv...
22040,tEWZp2wIkNmOblKgZr32lA,Wawa,3260 New Rodgers Rd,Bristol,PA,2.5,8,"{'Alcohol': 'u'none'', 'RestaurantsAttire': 'u...","Food, Convenience Stores, Fast Food, Automotiv...",service really gone downhill staff rude ordere...
41927,Hx7GFPYqTL6Ws11UJZZwBQ,Wawa,1083 East Lancaster Ave,Downingtown,PA,3.5,5,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Food, Sandwiches, Coffee & Tea, Restaurants, C...",great sandwiches always ask old bay seasoning ...
